In [1]:
import os
from subprocess import getoutput
import numpy as np

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# The GPU id to use, "0" to  "7"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
#!pip install torchvision
#!pip install torchsummary

In [4]:
import albumentations 
from albumentations.pytorch import ToTensorV2

In [5]:
import sys
sys.path.append("/Users/krithiga.r/Documents/eva6/pytorch-lib/")
print(sys.path)

['/Users/krithiga.r', '/Users/krithiga.r/opt/anaconda3/lib/python38.zip', '/Users/krithiga.r/opt/anaconda3/lib/python3.8', '/Users/krithiga.r/opt/anaconda3/lib/python3.8/lib-dynload', '', '/Users/krithiga.r/.local/lib/python3.8/site-packages', '/Users/krithiga.r/opt/anaconda3/lib/python3.8/site-packages', '/Users/krithiga.r/opt/anaconda3/lib/python3.8/site-packages/aeosa', '/Users/krithiga.r/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions', '/Users/krithiga.r/.ipython', '/Users/krithiga.r/Documents/eva6/pytorch-lib/']


In [6]:
from woollylib import main
from woollylib.dataset import get_cifar_loader
from woollylib.utils.transform import get_transform
from woollylib.utils.utils import get_device

In [7]:
###where should these go? I have a copy in main.py

batch_size = 64
ricap_beta = 0.4
class_map = {
    'PLANE': 0,
    'CAR': 1,
    'BIRD': 2,
    'CAT': 3,
    'DEER': 4,
    'DOG': 5,
    'FROG': 6,
    'HORSE': 7,
    'SHIP': 8,
    'TRUCK': 9
}

# Enable or disable visualizations
show_summary = True
show_dataset_analyze = True

In [8]:
from woollylib.main import train_profile, normalize
use_cuda, device = get_device()

In [9]:
train_loader, test_loader = get_cifar_loader(get_transform(train_profile), get_transform(normalize), batch_size=batch_size, use_cuda=use_cuda)


PermissionError: [Errno 13] Permission denied: '../data'

In [ ]:
epochs = 30
lr = 0.05
max_lr = 0.5
steps_per_epoch = len(train_loader)
dropout = False
drop_ratio = 0.1
use_l1 = False
lambda_l1 = 1e-7
momentum = 0.9
weight_decay = 0.000125
weight_decay = weight_decay/batch_size

norm='ln'
use1x1=True
ctype='depthwise_seperable'
base_channels=16
repetations=1
usedilation=False
use_skip=False
train_function=train
test_function=test

print("Using Device:", device)
print("Epochs:", epochs)
print("Lr:", lr)
print("Max Lr:", max_lr)
print("Batch Size:", batch_size)
print("Dropout:", dropout)
print("Dropout Ratio:", drop_ratio)
print("Momentum:", momentum)
print("Weight Decay:", weight_decay)
print("Use L1:", use_l1)
print("L1 Lambda:", lambda_l1)
print("RICAP Beta", ricap_beta)

In [ ]:
if show_dataset_analyze:
    # print class scale
#     print_class_scale(train_loader, class_map)
    # print data samples
#     print_samples_native(train_loader, class_map)
    print_samples(train_loader, class_map)

In [ ]:
from woollylib.main import optimizer, criteria, schedule, custom_scheduler

In [ ]:
### To change:

ctrain = train_function(use_l1=use_l1, lambda_l1=lambda_l1)

# Create model instance based on parameter which one to use
model = resNet18(
    image=(32, 32), 
    ctype=ctype, 
    use1x1=use1x1, 
    base_channels=base_channels, 
    layers=repetations, 
    drop_ratio=drop_ratio, 
    usedilation=usedilation, 
    use_skip=use_skip
).apply(initialize_weights).to(device)

# Create optimizer instance based on hyper parameters


# Create Pytorch One Cycle scheduler instance
# pytorch_scheduler = one_cycle_lr_pt(
#     optimizer, 
#     lr=lr, 
#     max_lr=max_lr, 
#     steps_per_epoch=steps_per_epoch, 
#     epochs=epochs, 
#     anneal_strategy='linear'
# )



# Create instance of trainer with all params
trainer = Training(
    model,
    optimizer,
    criteria,
    custom_scheduler,
    ctrain,
    test_function,
    train_loader,
    test_loader,
    lr,
    epochs,
    device,
    dropout
)

if show_summary:
    print_summary(model, input_size=(3, 32, 32))
#     print_modal_summary(trainer.model)
# Run trainer
trainer.run()